In [45]:
# T2-2. Pima Indians Diabetes (Classification) / 당뇨병
# 이상치 처리 (Glucose, BloodPressure, SkinThickness, Insulin, BMI가 0인 값)
# 풀이

# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./data/pima_diabetes.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Outcome')

# X_train.shape, X_test.shape, y_train.shape, y_test.shape

# EDA
# (614, 9) (154, 9) (614, 2) (154, 2)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 0 0 0 0
# print(X_train.isnull().sum()) 
# print(X_test.isnull().sum())
# print(y_train.isnull().sum())
# print(y_test.isnull().sum())

# print(X_train.head(3))
# print(y_train.head(10)) # Classifier

# 결측치 처리
# pass

# 이상치 처리
df = pd.concat([X_train, X_test])
# print(df.shape)

cond1 = df['Glucose'] == 0
cond2 = df['BloodPressure'] == 0
cond3 = df['SkinThickness'] == 0
cond4 = df['Insulin'] == 0
cond5 = df['BMI'] == 0
# print(df[cond1])

def func(x):
    if x == 0:
        x = None
    else:
        x = x
    return x

df['Glucose'] = df['Glucose'].apply(lambda x: func(x))
df['BloodPressure'] = df['BloodPressure'].apply(lambda x: func(x))
df['SkinThickness'] = df['SkinThickness'].apply(lambda x: func(x))
df['Insulin'] = df['Insulin'].apply(lambda x: func(x))
df['BMI'] = df['BMI'].apply(lambda x: func(x))
# print(df[df['Glucose'].isnull()])

# print(df.describe())
# print(df.isnull().sum())
df['Glucose'] = df['Glucose'].fillna(df['Glucose'].mean())
df['BloodPressure'] = df['BloodPressure'].fillna(df['BloodPressure'].mean())
df['SkinThickness'] = df['SkinThickness'].fillna(df['SkinThickness'].mean())
df['Insulin'] = df['Insulin'].fillna(df['Insulin'].mean())
df['BMI'] = df['BMI'].fillna(df['BMI'].mean())
# print(df.isnull().sum())

# 스케일링
df = df.set_index('id')
y_train = y_train.set_index('id')
y_test = y_test.set_index('id')
# print(df.head(3))
# print(df.describe())

from sklearn.preprocessing import minmax_scale

cols = df.columns
for col in cols:
    df[col] = minmax_scale(df[col])
# print(df.describe())

X_train = df[:X_train.shape[0]]
X_test = df[X_train.shape[0]:]

# train_test_split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2022)
# print(X_train.shape, X_val.shape, y_train.shape, y_val.shape)
# print(X_train.head(3))
# print(y_train.head(3))

# 모델 생성
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=2022)

# 모델 fit
model.fit(X_train, y_train)

# 성능 평가
from sklearn.metrics import roc_auc_score
print(model.score(X_val, y_val))
print(roc_auc_score(model.predict(X_val), y_val))

# predict
pred = model.predict(X_test)

# 제출
# print(X_test.head())
submit = pd.DataFrame()
submit['cust_id'] = X_test.index
submit['gender'] = pred

submit.to_csv('pima_pred.csv', index=False)

submit = pd.read_csv('pima_pred.csv')
# print(submit.head(3))

print('='*20)
print(model.score(X_test, y_test))
print(roc_auc_score(pred, y_test))


0.7073170731707317
0.7131327953044754
0.7532467532467533
0.7433479991619526


C:\Users\hjk70\AppData\Local\Temp\ipykernel_22120\825075829.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)
